In [1]:
import pandas as pd
from os.path import dirname, realpath, join, isfile
from urlparse import urlparse
from nb_utils import (get_ps1_or_ipaddress, read_pcap_fields_from_txts,
                   download_roku_channel_details, ROKU_MACS)

In [2]:
import seaborn as sns
%matplotlib inline
sns.set(color_codes=True)
sns.set(rc={'figure.figsize':(16,9)})

In [3]:
ROOT_PROJ_DIR = dirname(dirname(dirname(dirname(realpath('__file__')))))
DATA_DIR = join(ROOT_PROJ_DIR, "data/pcap_analysis/roku-channel-surfer/2018-09-27")
HTTP_REQS_DIR = join(DATA_DIR, "http")
DNS_QUERIES_DIR = join(DATA_DIR, "dns")
HTTP_CSV = join(DATA_DIR, "roku-2019-02-04-requests.csv")
HTTP_CSV2 = join(DATA_DIR, "roku-2018-09-27-requests.csv")

### Load channel details
- We only channels details to add helper columns to request dataframe
- Once we build and store the request details, we don't actually need to do this

In [4]:
channel_df = download_roku_channel_details()
channel_df.head()

,_category,_scrape_ts,accessCode,desc,name,payment,price,rankByWatched,rating,thumbnail
id,,,,,,,,,,
12,movies-tvs,1537037970,101D4DEA,Watch TV shows and movies recommended just for...,Netflix,None,0,1,75.890121,http://channels.roku.com/images/f5e60d25f2c944...
2285,movies-tvs,1537037970,hulu,Stream all your TV in one place with Hulu. Get...,Hulu,None,0,3,74.931961,https://image.roku.com/developer_channels/prod...
13,movies-tvs,1537037970,8C42649C,Prime members can watch thousands of movies an...,Prime Video,None,0,4,73.552605,https://image.roku.com/developer_channels/prod...
46041,movies-tvs,1537037970,SlingTV,"Sling TV is Live TV, Only Better. The #1 Live ...",Sling TV,None,0,5,68.119362,https://image.roku.com/developer_channels/prod...
23048,movies-tvs,1537037970,twctv,"With Spectrum TV, you can enjoy up to 250 live...",Spectrum TV,None,0,6,64.740654,https://image.roku.com/developer_channels/prod...


## Load HTTP request details
- Load the request details from the CSV or build from scratch

In [5]:
df = pd.read_csv(HTTP_CSV, sep='\t', encoding='utf-8')

In [6]:
df.head()

,channel_id,start_ts,command,select_idx,eth_src,ip_dst,req_method,protocol,url,channel_name,domain,host,rank,category
0,71530,1549020784,NaN,NaN,NaN,NaN,GET,http,http://vhx.imgix.net/tsl/assets/e3d507f1-1ab5-...,Tribeca Shortlist,imgix.net,vhx.imgix.net,2623,movies-tvs
1,71530,1549020784,NaN,NaN,NaN,NaN,GET,http,http://vhx.imgix.net/tsl/assets/ab02c785-7fb5-...,Tribeca Shortlist,imgix.net,vhx.imgix.net,2623,movies-tvs
2,71530,1549020784,NaN,NaN,NaN,NaN,GET,http,http://vhx.imgix.net/tsl/assets/1238048f-acd8-...,Tribeca Shortlist,imgix.net,vhx.imgix.net,2623,movies-tvs
3,71530,1549020784,NaN,NaN,NaN,NaN,GET,http,http://vhx.imgix.net/tsl/assets/67dd5360-fe70-...,Tribeca Shortlist,imgix.net,vhx.imgix.net,2623,movies-tvs
4,71530,1549020784,NaN,NaN,NaN,NaN,GET,http,http://vhx.imgix.net/tsl/assets/f37df523-a3e1-...,Tribeca Shortlist,imgix.net,vhx.imgix.net,2623,movies-tvs


### HTTPS Requests

In [7]:
#df.drop_duplicates(subset=["channel_id", "protocol"]).\
#    groupby(["protocol"]).size().reset_index(name="# protocol").\
#    sort_values(by=['# protocol'], ascending=False).head(10)
df_https = df.loc[df['protocol'] == 'https']
ch_no = df_https.channel_id.nunique()
print "Intercepted HTTPS traffic on %d channels" % (ch_no)
#df.loc[df['protocol'] == 'https'].drop_duplicates(subset=["channel_id"]).sort_values(by=['channel_id']).head(10)

Intercepted HTTPS traffic on 123 channels


### HTTP requests - overview

In [8]:
print "%d requests to %d distinct domains %d distinct hosts from %s distinct channels" %\
    (len(df), df.domain.nunique(), df.host.nunique(), df.channel_id.nunique())

42340 requests to 417 distinct domains 730 distinct hosts from 723 distinct channels


### Top HTTPS domains MITMed appearing on multiple channels
- domains contacted by most channels

In [9]:
df_https.drop_duplicates(subset=["channel_id", "domain"]).\
    groupby(["domain"]).size().reset_index(name="# channels").\
    sort_values(by=['# channels'], ascending=False).head(10)

,domain,# channels
71,vimeo.com,40
72,vimeocdn.com,25
58,springserve.com,11
70,uplynk.com,11
76,wordpress.com,5
42,nbcuni.com,5
53,scorecardresearch.com,5
3,akamaized.net,4
43,nick.com,3
46,pbs.org,3


### Top domains in terms of recurrence

In [10]:
df_https.drop_duplicates(subset=["channel_id", "url"]).\
    groupby(["domain"]).size().reset_index(name="# domains").\
    sort_values(by=['# domains'], ascending=False).head(10)

,domain,# domains
72,vimeocdn.com,407
77,youtube.com,219
76,wordpress.com,116
70,uplynk.com,58
71,vimeo.com,55
58,springserve.com,50
46,pbs.org,32
37,movieweb.com,26
36,midnightpulp.com,23
12,cocoro.tv,22


### Channels contacting the most number of unique HTTPS domains

In [11]:
df_n_domains_by_channel = df_https.drop_duplicates(subset=["channel_name", "domain", "rank"]).\
    groupby(["channel_name", "rank"]).size().reset_index(name="# domains").\
    sort_values(by=['# domains'], ascending=False)
df_n_domains_by_channel.head(10)

,channel_name,rank,# domains
0,8ctave HD,936,22
121,Zypy.TV,773,3
106,Timeless TV Westerns,1161,3
77,NBC,33,3
116,WatchFreeHorrorFlix,716,3
83,PeachFlicks,3001,2
101,The Essential Sci-Fi Channel,959,2
97,Spud's Trailer Trash,1896,2
95,Six Gun Cinema,735,2
93,Sci-fi Classic TV Shows,2049,2
